# Système de gestion d’emprunt dans une librairie

In [1]:
# Importation
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import datediff, to_date
import os, shutil

In [2]:
# Instantiation
spark = SparkSession.builder \
                    .master("local") \
                    .appName("FolksDF") \
                    .getOrCreate()

**Question 0 - Créer les données.**


In [3]:
l1 = [('07890','Jean Paul Sartre'),('05678','Pierre de Ronsard')]
rdd1 = spark.sparkContext.parallelize(l1) #On obtient un rdd
Author = rdd1.toDF(['aid','name']) #Transformation en dataframe
Author.createOrReplaceTempView('AuthorSQL') #Pour les requêtes SQL
Author.show()

+-----+-----------------+
|  aid|             name|
+-----+-----------------+
|07890| Jean Paul Sartre|
|05678|Pierre de Ronsard|
+-----+-----------------+



In [4]:
l2 = [("0001","L'existentialisme est un humanisme","Philosophie"),("0002","Huis clos. Suivi de Les Mouches","Philosophie"),("0003","Mignonne allons voir si la rose","Poème"),("0004","Les Amours","Poème")]
rdd2 = spark.sparkContext.parallelize(l2) 
book = rdd2.toDF(['bid','title','category']) 
book.createOrReplaceTempView('bookSQL') 
book.show()

+----+--------------------+-----------+
| bid|               title|   category|
+----+--------------------+-----------+
|0001|L'existentialisme...|Philosophie|
|0002|Huis clos. Suivi ...|Philosophie|
|0003|Mignonne allons v...|      Poème|
|0004|          Les Amours|      Poème|
+----+--------------------+-----------+



In [5]:
l3 = [('S15','toto','Math'),('S16','popo','Eco'),('S17','fofo','Mécanique')]
rdd3 = spark.sparkContext.parallelize(l3) 
Student = rdd3.toDF(['sid','sname','dept']) 
Student.createOrReplaceTempView('StudentSQL') 
Student.show()

+---+-----+---------+
|sid|sname|     dept|
+---+-----+---------+
|S15| toto|     Math|
|S16| popo|      Eco|
|S17| fofo|Mécanique|
+---+-----+---------+



In [6]:
l4 = [('07890','0001'),('07890','0002'),('05678','0003'),('05678','0004')]
rdd4 = spark.sparkContext.parallelize(l4) 
write = rdd4.toDF(['aid','bid']) 
write.createOrReplaceTempView('writeSQL') 
write.show()

+-----+----+
|  aid| bid|
+-----+----+
|07890|0001|
|07890|0002|
|05678|0003|
|05678|0004|
+-----+----+



In [7]:
l5 = [('S15','0003','02-01-2020','01-02-2020'),('S15','0002','13-06-2020','null'),('S15','0001','13-06-2020','13-10-2020'),('S16','0002','24-01-2020','24-01-2020'),('S17','0001','12-04-2020','01-07-2020')]
rdd5 = spark.sparkContext.parallelize(l5) 
borrow = rdd5.toDF(['sid','bid','checkout-time','return-time']) 
borrow.createOrReplaceTempView('borrowSQL') 
borrow.show()

+---+----+-------------+-----------+
|sid| bid|checkout-time|return-time|
+---+----+-------------+-----------+
|S15|0003|   02-01-2020| 01-02-2020|
|S15|0002|   13-06-2020|       null|
|S15|0001|   13-06-2020| 13-10-2020|
|S16|0002|   24-01-2020| 24-01-2020|
|S17|0001|   12-04-2020| 01-07-2020|
+---+----+-------------+-----------+



**Question 1 - Trouver les titres de tous les livres que l'étudiant sid='S15' a emprunté.**



In [8]:
### SQL
spark.sql(''' select sid, title
            from bookSQL 
            join borrowSQL
            on bookSQL.bid = borrowSQL.bid
            where sid='S15' ''').show()

+---+--------------------+
|sid|               title|
+---+--------------------+
|S15|Huis clos. Suivi ...|
|S15|Mignonne allons v...|
|S15|L'existentialisme...|
+---+--------------------+



In [9]:
### DSL
book.join(borrow,['bid']) \
    .select('sid','title') \
    .filter(F.col('sid')=='S15') \
    .show()

+---+--------------------+
|sid|               title|
+---+--------------------+
|S15|Huis clos. Suivi ...|
|S15|Mignonne allons v...|
|S15|L'existentialisme...|
+---+--------------------+



**Question 2 - Trouver les titres de tous les livres qui n'ont jamais été empruntés par un étudiant.**


In [10]:
### SQL
spark.sql(''' select req.bid, bookSQL.title 
            from (select bid 
                from bookSQL
                where bid not in (select bid from borrowSQL) ) req
            join bookSQL on bookSQL.bid = req.bid ''').show()

+----+----------+
| bid|     title|
+----+----------+
|0004|Les Amours|
+----+----------+



In [11]:
### DSL
book.join(borrow,'bid', how ='left') \
    .select('bid','title') \
    .filter(F.col("checkout-time").isNull()) \
    .show() 

+----+----------+
| bid|     title|
+----+----------+
|0004|Les Amours|
+----+----------+



**Question 3 - Trouver tous les étudiants qui ont emprunté le livre bid=’002’**


In [12]:
### SQL
spark.sql(''' select bid, borrowSQL.sid, sname
            from borrowSQL 
            join StudentSQL
            on StudentSQL.sid = borrowSQL.sid
            where bid='0002' ''').show()

+----+---+-----+
| bid|sid|sname|
+----+---+-----+
|0002|S16| popo|
|0002|S15| toto|
+----+---+-----+



In [13]:
### DSL
borrow.join(Student,['sid']) \
    .select('bid','sid','sname') \
    .filter(F.col('bid')=='0002') \
    .show()

+----+---+-----+
| bid|sid|sname|
+----+---+-----+
|0002|S16| popo|
|0002|S15| toto|
+----+---+-----+



**Question 4 - Trouver les titres de tous les livres empruntés par des étudiants en Mécanique (département Mécanique)**


In [14]:
### SQL
spark.sql(''' select req.dept, bookSQL.title 
            from (select borrowSQL.sid, bid, dept
                from borrowSQL
                join StudentSQL 
                on borrowSQL.sid = StudentSQL.sid
                where dept='Mécanique' ) req
            join bookSQL on bookSQL.bid = req.bid ''').show()

+---------+--------------------+
|     dept|               title|
+---------+--------------------+
|Mécanique|L'existentialisme...|
+---------+--------------------+



In [15]:
### DSL
borrow.join(Student,['sid']) \
    .join(book,['bid']) \
    .select('dept','title') \
    .filter(F.col('dept')=='Mécanique')\
    .show()

+---------+--------------------+
|     dept|               title|
+---------+--------------------+
|Mécanique|L'existentialisme...|
+---------+--------------------+



**Question 5 - Trouver les étudiants qui n’ont jamais emprunté de livre.**


In [16]:
### SQL
spark.sql(''' select req.sid, StudentSQL.sname
            from (select sid 
                from StudentSQL
                where sid not in (select sid from borrowSQL) ) req
            join StudentSQL on StudentSQL.sid = req.sid ''').show()

+---+-----+
|sid|sname|
+---+-----+
+---+-----+



Tous les étudiants ont emprunté au moins un livre.

In [17]:
### DSL
Student.join(borrow,'sid', how ='left') \
    .select('sid','sname') \
    .filter(F.col("checkout-time").isNull()) \
    .show() 

+---+-----+
|sid|sname|
+---+-----+
+---+-----+



**Question 6 - Déterminer l’auteur qui a écrit le plus de livres.**


In [18]:
### SQL
spark.sql(''' select W.aid, A.name, count(bid) as Nb_livres
            from writeSQL as W 
            join AuthorSQL as A
            on A.aid = W.aid
            group by W.aid, A.name
            order by Nb_livres desc''').show()

+-----+-----------------+---------+
|  aid|             name|Nb_livres|
+-----+-----------------+---------+
|05678|Pierre de Ronsard|        2|
|07890| Jean Paul Sartre|        2|
+-----+-----------------+---------+



In [19]:
### DSL
write.join(Author,'aid') \
    .select('aid','name','bid') \
    .groupBy('aid','name') \
    .agg(F.count('bid').alias('Nb_livres')) \
    .orderBy(F.col('Nb_livres').desc()) \
    .show() 

+-----+-----------------+---------+
|  aid|             name|Nb_livres|
+-----+-----------------+---------+
|05678|Pierre de Ronsard|        2|
|07890| Jean Paul Sartre|        2|
+-----+-----------------+---------+



**Question 7 -  Déterminer les personnes qui n’ont pas encore rendu les livres.**


In [20]:
### SQL
spark.sql(''' select borrowSQL.sid, sname, `return-time`
            from borrowSQL
            join StudentSQL
            on StudentSQL.sid = borrowSQL.sid
            where `return-time`='null' ''').show()

+---+-----+-----------+
|sid|sname|return-time|
+---+-----+-----------+
|S15| toto|       null|
+---+-----+-----------+



In [21]:
### DSL
borrow.join(Student,'sid') \
    .select('sid','sname','return-time') \
    .filter(F.col('return-time')=='null') \
    .show()

+---+-----+-----------+
|sid|sname|return-time|
+---+-----+-----------+
|S15| toto|       null|
+---+-----+-----------+



**Question 8 - Créer une nouvelle colonne dans la table borrow qui prend la valeur 1, si la durée d'emprunt est supérieur à 3 mois,  sinon 0.  (utiliser la fonction spark to_date) par la suite faire un export des données en CSV. dans un répertoire nommé contention (Attention pas de path en dur ! )**


In [22]:
###SQL
spark.sql('''select *,
                case 
                    when duree >= 90 then 1
                    else 0
                end as time
            from (select *, DATEDIFF(TO_DATE(`return-time`,"dd-MM-yyyy"),TO_DATE(`checkout-time`,"dd-MM-yyyy")) as duree
            from borrowSQL)''').show()

+---+----+-------------+-----------+-----+----+
|sid| bid|checkout-time|return-time|duree|time|
+---+----+-------------+-----------+-----+----+
|S15|0003|   02-01-2020| 01-02-2020|   30|   0|
|S15|0002|   13-06-2020|       null| null|   0|
|S15|0001|   13-06-2020| 13-10-2020|  122|   1|
|S16|0002|   24-01-2020| 24-01-2020|    0|   0|
|S17|0001|   12-04-2020| 01-07-2020|   80|   0|
+---+----+-------------+-----------+-----+----+



In [23]:
###DSL
Table = borrow.withColumn("duree",
                 datediff(to_date("return-time","dd-MM-yyyy"),
                                  to_date("checkout-time","dd-MM-yyyy"))/30)\
.withColumn("duree",F.when(F.col("duree")>=3,1).otherwise(0))
Table.show()

+---+----+-------------+-----------+-----+
|sid| bid|checkout-time|return-time|duree|
+---+----+-------------+-----------+-----+
|S15|0003|   02-01-2020| 01-02-2020|    0|
|S15|0002|   13-06-2020|       null|    0|
|S15|0001|   13-06-2020| 13-10-2020|    1|
|S16|0002|   24-01-2020| 24-01-2020|    0|
|S17|0001|   12-04-2020| 01-07-2020|    0|
+---+----+-------------+-----------+-----+



In [24]:
#Exportation
if os.path.exists("contention"):
        shutil.rmtree("contention")
Table.write.csv('contention', sep = ';', header = 'true')

**Question 9 - Déterminer les livres qui n’ont jamais été empruntés.** 


In [25]:
### SQL
#solution 1
spark.sql(''' select bid
            from bookSQL
            EXCEPT
            select bid
            from borrowSQL ''').show()

+----+
| bid|
+----+
|0004|
+----+



In [26]:
#solution2
spark.sql('''select bid 
            from bookSQL
            where bid not in (select bid 
                            from  borrowSQL)''').show()

+----+
| bid|
+----+
|0004|
+----+



In [27]:
### DSL
book.join(borrow,'bid', how ='left') \
    .select('bid') \
    .filter(F.col("checkout-time").isNull()) \
    .show() 

+----+
| bid|
+----+
|0004|
+----+

